In [2]:
import pandas as pd
import numpy as np
import re
from datetime import datetime as dt
from scipy.spatial import distance
import scipy.stats as stats

%matplotlib inline

In [3]:
TIME_SAMPLE_FREQ = '30s'

In [4]:
df = pd.read_csv(".\\_generated\\wifi_1.data", index_col = False, header = None, low_memory = False, \
                 names = ['timestamp', 'uuid', 'bssid', 'ssid', 'capabilities', \
                          'freq0', 'freq1', 'chwidth', 'freq', 'level', 'operator', \
                          'time', 'venue', '802', 'passpoint'])

FileNotFoundError: [Errno 2] File b'.\\_generated\\wifi_1.data' does not exist: b'.\\_generated\\wifi_1.data'

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['timestamp'] = df['timestamp'].apply(lambda x: dt.strptime(x, '%d.%m.%Y_%H:%M:%S.%f'))

In [ ]:
df.index = pd.DatetimeIndex(df.timestamp)
df = df.sort_index()

In [ ]:
df = df.drop(['timestamp', 'chwidth', 'operator', 'venue', '802', 'time', 'ssid', 'capabilities', \
              'passpoint', 'freq0', 'freq1'], axis = 1)

In [ ]:
df.info()

In [ ]:
bssid_map = { bssid.replace(' ', ''): idx for bssid, idx in zip(df.bssid.unique(), range(len(df.bssid.unique()))) }

In [ ]:
df.bssid = df.bssid.apply(lambda x: str(x).replace(' ', ''))
df.level = df.level.apply(lambda x: str(x).replace(' ', ''))
df.freq = df.freq.apply(lambda x: str(x).replace(' ', ''))

In [ ]:
df['bssid_level'] = df[['bssid', 'level']].agg(','.join, axis=1)
df['count'] = 1

In [ ]:
def agg_string_join(col):
    col = col.apply(lambda x: str(x))
    return col.str.cat(sep = ',').replace(' ', '')

In [ ]:
def agg_bssid_col(col):
    array_len = len(bssid_map)
    array = np.zeros(array_len, dtype = 'float')
    def fill_array(x):
        tmp = x.split(',')
        bssid = tmp[0]
        level = float(tmp[1])
        array[bssid_map[bssid.replace(' ', '')]] = level
        return
        
    col.apply(lambda x: fill_array(x))
    return np.array2string(array, separator = ',').replace(' ', '')[1:-1]

In [ ]:
all_func_dicts_quantum = { 'freq': agg_string_join, 'level': agg_string_join, 'bssid_level' : agg_bssid_col, 'count' : 'sum' }

In [ ]:
df_quantum = df.groupby(['timestamp', 'uuid'], as_index=True).agg(all_func_dicts_quantum)

In [ ]:
df_quantum = df_quantum.reset_index()
df_quantum.index = pd.DatetimeIndex(df_quantum.timestamp)

In [ ]:
df_quantum = df_quantum[df_quantum['count'] != 0]

In [ ]:
df_conn = pd.read_csv(".\\_generated\\conn_wifi_5.data", index_col = False, header = None, low_memory = False, \
                 names = ['timestamp', 'uuid', 'stamp', 'bssid', '1', '2', '3', '4', '5', '6', 'level', '8'])

df_conn = df_conn.drop(df_conn.columns.difference(['bssid','timestamp', 'level']), axis = 1)
df_conn['timestamp'] = df_conn['timestamp'].apply(lambda x: dt.strptime(x, '%d.%m.%Y_%H:%M:%S.%f'))
df_conn.index = pd.DatetimeIndex(df_conn.timestamp)
df_conn = df_conn.sort_index()

In [ ]:
def get_level_from_row(row):
    bssid = df_conn.iloc[df_conn.index.get_loc(row.name, method = 'nearest')]['bssid']
    if str(bssid) == 'nan' or str(bssid) == 'null' or str(bssid) == '':
        return 0
    
    level = df_conn.iloc[df_conn.index.get_loc(row.name, method = 'nearest')]['level']
    time = df_conn.iloc[df_conn.index.get_loc(row.name, method = 'nearest')]['timestamp']
    return level if abs((time - row.name).total_seconds()) <= 10 else 0


df_quantum['conn_level'] = df_quantum.apply(lambda row: get_level_from_row(row), axis = 1)

In [ ]:
def string2array(string):
    try:
        array = np.fromstring(string, sep=',')
        return array
    except:
        return np.nan

def to_ones_array(array):
    try:
        array[array != 0] = 1
        return array
    except:
        return np.nan

def get_len(obj):
    try:
        length = len(obj)
        return length
    except:
        return np.nan

In [ ]:
def get_occured_nets_count(row, prev_col, curr_col):
    prev = to_ones_array(string2array(row[prev_col]))
    curr = to_ones_array(string2array(row[curr_col]))
    intersection = np.logical_and(curr, prev)
    diff = np.logical_and(curr, np.logical_not(intersection))
    
    if (np.count_nonzero(np.logical_or(prev, curr)) == 0):
        return 0
    
    return np.count_nonzero(diff) / np.count_nonzero(np.logical_or(prev, curr))

def get_disappeared_nets_count(row, prev_col, curr_col):
    prev = to_ones_array(string2array(row[prev_col]))
    curr = to_ones_array(string2array(row[curr_col]))
    intersection = np.logical_and(curr, prev)
    diff = np.logical_and(prev, np.logical_not(intersection))
    
    if (np.count_nonzero(np.logical_or(prev, curr)) == 0):
        return 0
    
    return np.count_nonzero(diff) / np.count_nonzero(np.logical_or(prev, curr))

def get_jaccard_index(row, prev_col, curr_col):
    prev = to_ones_array(string2array(row[prev_col]))
    curr = to_ones_array(string2array(row[curr_col]))
    return distance.jaccard(prev, curr)

def get_occur_speed(row, prev_col, curr_col):
    prev = to_ones_array(string2array(row[prev_col]))
    curr = to_ones_array(string2array(row[curr_col]))
    return np.linalg.norm(prev - curr) / np.sqrt(get_len(prev))
    
def get_level_speed(row, prev_col, curr_col):
    prev = string2array(row[prev_col])
    curr = string2array(row[curr_col])
    return np.linalg.norm(prev - curr) / np.sqrt(get_len(prev))

def calc_single_cols_in_window(df, col, new_col, window, func):
    def func_wrapper(func, row, prev_col, curr_col):
        delta = row.timestamp - row.prev_timestamp
        if pd.isnull(delta):
            delta = 0
        else:
            delta = abs(delta.total_seconds())
        if delta > 10 * 60:
            return np.nan
        else:
            return func(row, prev_col_name, col)
        
    new_cols = []
        
    for i in range(window):
        prev_col_name = "_".join(['prev', col, str(i + 1)])
        new_col_name = "_".join([new_col, str(i + 1)])
        
        df['prev_timestamp'] = df.timestamp.shift(i + 1)
        df[prev_col_name] = df[col].shift(i + 1)
        df[new_col_name] = df.apply(lambda row: func_wrapper(func, row, prev_col_name, col), axis = 1)
        df = df.drop(prev_col_name, axis = 1)
        df = df.drop('prev_timestamp', axis = 1)
        new_cols.append(new_col_name)
        
    df["_".join([new_col, 'mean'])] = df[new_cols].mean(axis = 1)
    df["_".join([new_col, 'median'])] = df[new_cols].median(axis = 1)
    df["_".join([new_col, 'var'])] = df[new_cols].var(axis = 1)
        
    return df

In [ ]:
WINDOW_SIZE = 3

occur_and_level_columns_map = [
    ("bssid_level", "occured_nets_count", WINDOW_SIZE, get_occured_nets_count),
    ("bssid_level", "disappeared_nets_count", WINDOW_SIZE, get_disappeared_nets_count),
    ("bssid_level", "jaccard_index", WINDOW_SIZE, get_jaccard_index), 
    ("bssid_level", "occur_speed", WINDOW_SIZE, get_occur_speed),
    ("bssid_level", "level_speed", WINDOW_SIZE, get_level_speed)
]

for (col, new_col, window, func) in occur_and_level_columns_map:
    df_quantum = calc_single_cols_in_window(df_quantum, col, new_col, window, func)

In [ ]:
def get_conn_level_speed(row, prev_col, curr_col):
    return row[curr_col] - row[prev_col]

In [ ]:
WINDOW_SIZE = 3

single_columns_map = [
    ("conn_level", "conn_level_speed", WINDOW_SIZE, get_conn_level_speed),
    ("count", "count_speed", WINDOW_SIZE, get_conn_level_speed)
]

for (col, new_col, window, func) in single_columns_map:
    df_quantum = calc_single_cols_in_window(df_quantum, col, new_col, window, func)

In [ ]:
def get_acceleration(row, prev_col, curr_col):
    return abs(row[curr_col] - row[prev_col])

In [ ]:
WINDOW_SIZE = 3

multi_speed_cols = ["occured_nets_count_mean", "jaccard_index_mean", "occur_speed_mean", "disappeared_nets_count_mean", 
                    "conn_level_speed_mean", "count_speed_mean"]

for col in multi_speed_cols:
    df_quantum = calc_single_cols_in_window(df_quantum, col, "_".join(["acceleration", col]), window, func)

In [ ]:
def agg_str(col):
#     all_freq = col.str.cat(sep=',')
    return string2array(col)

def str_mean(col):
    array = agg_str(col)
    if str(array) == 'nan':
        return 0 
    return np.mean(array)

# def str_var(col):
#     array = agg_str(col)
#     if str(array) == 'nan':
#         return 0 
#     return np.var(array)

# def str_median(col):
#     array = agg_str(col)
#     if str(array) == 'nan':
#         return 0 
#     return np.median(array)

# def str_skew(col):
#     array = agg_str(col)
#     if str(array) == 'nan':
#         return 0 
#     return stats.skew(array)

# def str_kurt(col):
#     array = agg_str(col)
#     if str(array) == 'nan':
#         return 0 
#     return stats.kurtosis(array)

def mean(col):
    return np.mean(col)

def var(col):
    return np.var(col)

def median(col):
    return np.median(col)

def skew(col):
    return stats.skew(col)

def kurt(col):
    return stats.kurtosis(col)

In [ ]:
df_quantum['freq'] = df_quantum.apply(lambda row: str_mean(row['freq']), axis = 1)
df_quantum['level'] = df_quantum.apply(lambda row: str_mean(row['level']), axis = 1)

In [ ]:
df_quantum = df_quantum.drop(['bssid_level', 'timestamp', 'uuid'], axis = 1)

In [ ]:
# string_common_cols = df_quantum.columns[1:3]
common_cols = df_quantum.columns[1:5]
speed_acc_cols = df_quantum.columns[5:]

# string_common_funcs_list = [str_mean, str_var, str_median, str_skew, str_kurt]
common_funcs_list = [mean, var, median, skew, kurt]
special_funcs_list = [mean, pd.DataFrame.mad, skew]

common_cols_map = { col : common_funcs_list for col in common_cols }
# string_common_cols_map = { col : string_common_funcs_list for col in string_common_cols }
speed_acc_cols_map = { col : special_funcs_list for col in speed_acc_cols }

agg_dict = common_cols_map
# agg_dict.update(string_common_cols_map)
agg_dict.update(speed_acc_cols_map)

In [ ]:
df_quantum[speed_acc_cols] = df_quantum[speed_acc_cols].apply(pd.to_numeric)

In [ ]:
df_sampling = df_quantum.groupby(pd.Grouper(freq = TIME_SAMPLE_FREQ)).agg(agg_dict)

In [ ]:
df_rolling = df_quantum.rolling(TIME_SAMPLE_FREQ, min_periods = 1, center = False).agg(agg_dict)

In [ ]:
df_sampling.columns = ["_".join([str(high_level_name), str(low_level_name)]) \
                       for (high_level_name, low_level_name) in df_sampling.columns.values]

df_rolling.columns = ["_".join([str(high_level_name), str(low_level_name)]) \
                      for (high_level_name, low_level_name) in df_rolling.columns.values]

In [ ]:
df_sampling = df_sampling.dropna()
df_sampling = df_sampling.fillna(0)

df_rolling = df_rolling.dropna()
df_rolling = df_rolling.fillna(0)

In [ ]:
# df_rolling['day_time'] = df_rolling.apply(lambda x: x.name.hour + x.name.minute / 60 , axis = 1)
# df_sampling['day_time'] = df_sampling.apply(lambda x: x.name.hour + x.name.minute / 60 , axis = 1)

In [ ]:
df_sampling.to_csv(".\\_datasets\\5s\\wifi_sampling_dataset_5.csv")
df_rolling.to_csv(".\\_datasets\\5s\\wifi_rolling_dataset_5.csv")